# Lab-1 基于模型的协同过滤算法的实现

## 基本要求
- 基于UV分解，建立协同过滤模型（矩阵分解的代码要自己编写）
- 在user_artist_data中，预留20%的数据，作为验证集
- 计算模型对验证集进行预测的结果的RMSE

## 数据集
数据集路径均为`./*.txt`
- `user_artist_data.txt`：2420万条用户播放艺术家歌曲次数
- `artist_data.txt`: 160+万个艺术家的ID和名字
- `artist_alias.txt`: 拼写错误的艺术家ID（变体）到该艺术家的规范ID的映射关系（19万条记录）

In [1]:
import numpy as np
import pandas as pd

# 由于数据中存在使用分隔符不统一的问题，所以不能直接使用pandas进行数据载入
# 故首先进行数据格式问题的处理

In [2]:
import os

print('user_artist_data.txt 中数据行数：')
print(os.popen('cat user_artist_data.txt | wc -l').read())
print('user_artist_data.txt 中数据格式')
print(os.popen('head -5 user_artist_data.txt').read())
print('user_artist_data.txt 中\" \"的个数')
print(os.popen('grep -o \' \' user_artist_data.txt | wc -l').read())

user_artist_data.txt 中数据行数：
 24296858

user_artist_data.txt 中数据格式
1000002 1 55
1000002 1000006 33
1000002 1000007 8
1000002 1000009 144
1000002 1000010 314

user_artist_data.txt 中" "的个数
 48593716



可以看到，在`user_artist_data.txt`共有2400万余条数据，并且使用空格作为分隔符。如果数据分割正确，则文件中的空格数应恰好为行数的2倍，此处恰好满足。**故在`user_artist_data.txt`中数据分割正确**。

In [3]:
print('artist_data.txt 中数据的行数')
print(os.popen('cat artist_data.txt | wc -l').read())
print('artist_data.txt 中数据格式')
print(os.popen('head -5 artist_data.txt').read())
print('artist_data.txt 中\'\\t\'的个数')
print(os.popen('grep \'\t\' artist_data.txt | wc -l').read())

artist_data.txt 中数据的行数
 1848579

artist_data.txt 中数据格式
1134999	06Crazy Life
6821360	Pang Nakarin
10113088	Terfel, Bartoli- Mozart: Don
10151459	The Flaming Sidebur
6826647	Bodenstandig 3000

artist_data.txt 中'\t'的个数
 1848281



可以看到，在`artist_data.txt`中共有184万余条数据，每条数据有两个数据域，分别表示为`artist_id`和`aritst_name`，并使用`\t`作为分割符。如果数据分割正确，则文件中的`\t`数量应恰好为行数，但此处可以发现有部分行缺少分隔符，所以不能直接进行数据的载入，需要对缺失部分进行处理。

由于输入格式的问题，此处不在笔记本中记录筛选的过程（主要在于`\t`和`\n`在notebook中的处理过于繁琐，直接利用`shell`脚本会更直接一些）。
> `shell`命令为`grep -Ev '^[0-9]+<Ctrl-v><tab>[^<Ctrl-v><tab>^M]$ > artist_wrong_format_data.txt'`，用于查看错误的格式的数据。

In [4]:
print('错误格式的数据共有', os.popen('cat artist_wrong_format_data.txt | wc -l').read())
print(os.popen('head artist_wrong_format_data.txt').read())

错误格式的数据共有      516

10024027	Rio Natsuki
Aya Hisakawa
Aya Sakaguchi
Miki Itou
Youko Sawami
Taeko Kawada
}4
10113424	-0BY,兤ﾁ・Eﾎ~・,
0sH,ｮ,80ｮP,ｰ蔑
1256670	Einojuhani Rautavaara
Helsinki Philharmonic Orchestra - Leif Segerstam,   Kari Jussila - Organ



根据前面的信息可以看到，错误主要有以下几种:
1. 缺少`artist_id`信息域，导致无法用`\t`分割数据
2. 信息中存在`\n`换行符，导致直接利用`pandas`导入数据，无法正常划分
3. 信息中存在一条内有多个`\t`的情况

In [5]:
# print(os.popen('grep -Ev \'^[0-9]+\t[^\t]+$\' artist_data.txt').read())
# print(os.popen('grep -Ev \'\^[0-9]+\t[\^\t\n]+$\' artist_data.txt > artist_wrong_format_data.txt').read())
# 去除掉错误的信息后，可以正常使用的`artist_data`数据存放在`artist_correct_format_data.txt`
print('artist_correct_format_data.txt 中数据行数：')
print(os.popen('cat artist_correct_format_data.txt | wc -l').read())
print('artist_correct_format_data.txt 中数据格式')
print(os.popen('head -5 artist_correct_format_data.txt').read())

artist_correct_format_data.txt 中数据行数：
 1848063

artist_correct_format_data.txt 中数据格式
1134999	06Crazy Life
6821360	Pang Nakarin
10113088	Terfel, Bartoli- Mozart: Don
10151459	The Flaming Sidebur
6826647	Bodenstandig 3000



In [6]:
print('artist_alias.txt 中数据的行数')
print(os.popen('cat artist_alias.txt | wc -l').read())
print('artist_data.txt 中数据格式')
print(os.popen('head -5 artist_alias.txt').read())
print('artist_alias.txt 中\'\\t\'的个数')
print(os.popen('grep \'\t\' artist_alias.txt | wc -l').read())

artist_alias.txt 中数据的行数
  193027

artist_data.txt 中数据格式
1092764	1000311
1095122	1000557
6708070	1007267
10088054	1042317
1195917	1042317

artist_alias.txt 中'\t'的个数
  193027



可以看到，对于`artist_alias.txt`数据而言，其使用`\t`进行分割，且共有19万余条数据。恰好数据中`\t`的数量与数据行数相同，可以认为数据分割没有错误。

In [12]:
# 查看数据规模，并验证数据集路径正确
user_artist_data = pd.read_csv('./user_artist_data.txt', sep=' ', header=None)
user_artist_data.columns = ['user_id', 'song_id', 'times']
user_artist_data = user_artist_data.astype({'user_id':str, 'song_id':str, 'times':int})
user_artist_data['id'] = list(zip(user_artist_data['user_id'], user_artist_data['song_id']))
user_artist_data = user_artist_data.drop(['user_id', 'song_id'], axis=1)
user_artist_data = user_artist_data.set_index('id')
print("user_artist_data info")
print(user_artist_data.info())
print(user_artist_data.describe())
print('-' * 30)

artist_data = pd.read_csv('./artist_correct_format_data.txt', sep='\t', header=None)
artist_data.columns = ['artist_id', 'artist_name']
artist_data = artist_data.astype({'artist_id':str, 'artist_name':str})
print('artist_data info')
print(artist_data.info())
print(artist_data.describe())
print('-' * 30)

artist_alias = pd.read_csv('./artist_alias.txt', sep='\t', header=None)
artist_alias.columns = ['wrong_artist_id', 'correct_artist_id']
artist_alias = artist_alias.astype({'wrong_artist_id':str, 'correct_artist_id':str})
print('artist_alias info')
print(artist_alias.info())
print(artist_alias.describe())

user_artist_data info
<class 'pandas.core.frame.DataFrame'>
Index: 24296858 entries, (1000002, 1) to (9875, 9973009)
Data columns (total 1 columns):
times    int64
dtypes: int64(1)
memory usage: 370.7+ MB
None
              times
count  2.429686e+07
mean   1.529576e+01
std    1.539153e+02
min    1.000000e+00
25%    1.000000e+00
50%    3.000000e+00
75%    9.000000e+00
max    4.397710e+05
------------------------------
artist_data info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1829523 entries, 0 to 1829522
Data columns (total 2 columns):
artist_id      object
artist_name    object
dtypes: object(2)
memory usage: 27.9+ MB
None
       artist_id        artist_name
count    1829523            1829523
unique   1829523            1829228
top      1067211  Weird Al Yankovic
freq           1                  4
------------------------------
artist_alias info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193027 entries, 0 to 193026
Data columns (total 2 columns):
wrong_artist_id      

In [ ]:
user_artist_data = pd.read_csv('./user_artist_data.txt', sep=' ', header=None)
user_artist_data.columns = ['user_id', 'song_id', 'times']
user_artist_data = user_artist_data.astype({'user_id':str, 'song_id':str, 'times':int})

In [8]:
user = set(user_artist_data['user_id'])
print('用户数量',len(user))

artist = set(user_artist_data['song_id'])
print('艺术家数量', len(artist))

用户数量 148111
艺术家数量 1631028


根据上面单元格的信息，*用户数量14万，艺术家数量为160万*，我们简单估计一下：
- 如果使用稠密表示，所需内存为`1.4e5 * 1.6e6 * 8/ (1e9) ≈ 2000GB`
显然以上数据不能再内存中稠密计算，因此需要进行稀疏表示。

In [9]:
# 由于user_id和artist_id均不是连续表示的，所以为了方便矩阵操作
# 增加id到index之间的映射，用于数据表示的离散化
user_to_index = {x:y for x,y in enumerate(user)}
artist_to_index = {x:y for x,y in enumerate(artist)}

In [10]:
# 使用20%作为验证集
from sklearn.model_selection import train_test_split

train_user_artist_data, test_user_artist_data = train_test_split(user_artist_data, test_size=0.2, random_state=3)
print(train_user_artist_data.shape)
print(test_user_artist_data.shape)

(19437486, 3)
(4859372, 3)


In [11]:
# M = train_user_artist_data.set_index(['user_id', 'song_id']).T.to_dict('list')
# print(M.get((user_to_index.get(1), artist_to_index.get(1)), None))
# print(train_user_artist_data.loc())
# empty.set_index(['from', 'to']).T.to_dict('list')

KeyboardInterrupt: 

In [ ]:
# 根据对user_artist_data的统计数据
# 平均听歌次数为15.36次，数据标准差为153.92
# 所以将效用矩阵元素表示为(x - mean) / std，x为听歌次数
M = np.full((len(user_to_index), len(artist_to_index)), np.nan)
# for data in train_user_artist_data:
#     M[user_to_index[data[0]]][artist_to_index[data[1]]] = data[2]
print(M[:10,:10])

In [ ]:
test_m = [[5, 2, 4, 4, 3], [3, 1, 2, 4, 1], [2, -1, 3, 1, 4], [2, 5, 4, 3, 5], [4, 4, 5, 4, -1]]
test_m = np.array(test_m)
n, m = test_m.shape
empty = pd.DataFrame(columns={'from', 'to', 'value'})
for i in range(n):
    for j in range(m):
        if(test_m[i][j] > 0):
            new = pd.DataFrame({"from":i,"to":j,"value":test_m[i][j]}, index=['0'])
            empty = empty.append(new, ignore_index=True)
print(empty)

In [ ]:
def rmse(M, U, V):
    n, d = U.shape
    d_1, m = V.shape
    assert d == d_1
    res = 0
    for i in range(n):
        for j in range(m):
            t = M.get((i, j), None)
            if t is None:
                continue
            p = 0
            for k in range(d):
                p = p + U[i][k] * V[k][j]
            res = res + (p - t[0]) * (p - t[0])
    return res

def UV_decomposition(M, n, m, d=2, epsilon=1e-3):
    U = np.ones((n, d))
    V = np.ones((d, m))
    time = 0
    rmse_last = rmse(M, U, V)
    while True:
        u_terms = True
        ui = uj = vi = vj = 0
        u_finished = v_finished = False
        terminated = rmse_last
        while True:
            print(time, rmse_last)
            time += 1
            if u_finished and v_finished:
                break
            if v_finished or u_terms:
                u_terms = False
                numerator = 0
                denominator = 0
                for j in range(m):
                    p = 0
                    for k in range(d):
                        if k != uj:
                            p = p + U[ui][k] * V[k][j]
                    m_rj = M.get((ui, j), None)
                    if m_rj is None:
                        continue
                    numerator = numerator + V[uj][j] * (m_rj[0] - p)
                    denominator = denominator + V[uj][j] * V[uj][j]
                last_u = U[ui][uj]
                U[ui][uj] = numerator * 1.0 / denominator
                rmse_this = rmse(M, U, V)
                if rmse_this < rmse_last:
                    rmse_last = rmse_this
                else:
                    U[ui][uj] = last_u
                #             print(U)
                uj += 1
                if uj >= d:
                    ui += 1
                    uj = 0
                    if ui >= n:
                        u_finished = True
            elif u_finished or not u_terms:
                u_terms = True
                numerator = 0
                denominator = 0
                for i in range(n):
                    p = 0
                    for k in range(d):
                        if k != vi:
                            p = p + U[i][k] * V[k][vj]
                    m_is = M.get((i, vj), None)
                    if m_is is None:
                        continue
                    numerator = numerator + U[i][vi] * (m_is[0] - p)
                    denominator = denominator + U[i][vi] * U[i][vi]
                last_v = V[vi][vj]
                V[vi][vj] = numerator * 1.0 / denominator
                #             print(V)
                rmse_this = rmse(M, U, V)
                if rmse_this < rmse_last:
                    rmse_last = rmse_this
                else:
                    V[vi][vj] = last_v
                vj += 1
                if vj >= m:
                    vi += 1
                    vj = 0
                    if vi >= d:
                        v_finished = True
        if abs(terminated - rmse_last) < epsilon:
            break
        else:
            terminated = rmse_last
    return U, V

In [ ]:
# UV_decomposition(empty, n=5, m=5)
d = empty.set_index(['from', 'to']).T.to_dict('list')
U, V = UV_decomposition(d, n=5, m=5)
# print(d)
# print(d[1, 2])
# print(d.get((2, 1), None))